# Museum Analysis

This notebook connects to the museum database and conducts simple analysis of the data.

## Imports

In [1]:
from os import environ
from dotenv import load_dotenv
from psycopg2 import connect, OperationalError
from psycopg2.extras import execute_values, RealDictCursor

from pipeline import get_db_connection

## Setup

In [17]:
load_dotenv()

conn = get_db_connection()

def run_sql_query(conn, sql_query: str):
    """Execute a given sql query for database connection"""

    with conn.cursor() as curr:
        curr.execute(sql_query)
        rows = curr.fetchall()
        return rows

### 1. What exhibition is most frequently visited?

In [3]:
sql_query = """
            SELECT e.exhibition_name, e.exhibition_id, COUNT(ri.rating_instance_id) AS instance_count
            FROM exhibition AS e
            JOIN rating_instance as ri
                ON e.exhibition_id = ri.exhibition_id
            GROUP BY e.exhibition_id, e.exhibition_name
            ORDER BY instance_count
            DESC
            ;
            """

data = run_sql_query(conn, sql_query)
data[0].get('exhibition_name') + " was the most frequently visited exhibition."

'Our Polluted World was the most frequently visited exhibition.'

### 2. What hour of the day has the most ratings?

In [4]:
sql_query = """
            SELECT EXTRACT(HOUR FROM ri.rating_created_at) AS hour,
                COUNT(*) as num_ratings
            FROM rating_instance AS ri
            GROUP BY hour
            ORDER BY num_ratings DESC
            ;
            """

data = run_sql_query(conn, sql_query)
str(data[0].get("hour")) + ":00."

'10:00.'

### 3. What is the average rating for each exhibition?

In [5]:
sql_query = """
            SELECT e.exhibition_name, ROUND(AVG(rt.rating_type_value), 2) as average_rating
            FROM exhibition AS e
            JOIN rating_instance AS ri
            ON e.exhibition_id = ri.exhibition_id
            JOIN rating_type AS rt
            ON ri.rating_type_id = rt.rating_type_id
            GROUP BY e.exhibition_name
            ;
            """

data = run_sql_query(conn, sql_query)
for exhibition in data:
    print(f"The average rating for {exhibition.get('exhibition_name')} is {exhibition.get('average_rating')}.")

The average rating for Adaptation is 1.93.
The average rating for Cetacean Sensations is 2.83.
The average rating for The Crenshaw Collection is 1.43.
The average rating for Our Polluted World is 1.22.
The average rating for Measureless to Man is 1.92.


### 4. What proportion of all 4+ ratings are given to exhibition 4?

In [6]:
sql_query = """
            SELECT e.exhibition_id, e.exhibition_name,
            SUM(CASE WHEN rt.rating_type_value > 3 THEN 1 ELSE 0 END) * 100 / COUNT(*) AS proportion
            FROM exhibition AS e
            JOIN rating_instance AS ri
            ON ri.exhibition_id = e.exhibition_id
            JOIN rating_type as rt
            ON rt.rating_type_id = ri.rating_type_id
            WHERE e.exhibition_id = 5
            GROUP BY e.exhibition_id;
            """

data = run_sql_query(conn, sql_query)
exhibition = data[0]
print(
    f"Proportion of 4+ ratings in Exhibition 4 ({exhibition.get('exhibition_name')}) = {exhibition.get('proportion')}%.")

Proportion of 4+ ratings in Exhibition 4 (Our Polluted World) = 7%.


### 5. Are positive ratings more frequent before or after 1pm?

In [7]:
sql_query = """
            SELECT
            ROUND(SUM(CASE WHEN EXTRACT(HOUR FROM ri.rating_created_at) < 13 THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1)
                AS positive_before_1pm,
            ROUND(SUM(CASE WHEN EXTRACT(HOUR FROM ri.rating_created_at) >= 13 THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1)
                AS positive_1pm_and_beyond
            FROM exhibition AS e
            JOIN rating_instance AS ri
            ON ri.exhibition_id = e.exhibition_id
            JOIN rating_type AS rt
            ON rt.rating_type_id = ri.rating_type_id
            WHERE rating_type_value >= 3
            ;
            """

data = run_sql_query(conn, sql_query)
before = int(data[0].get('positive_before_1pm'))
after = int(data[0].get('positive_1pm_and_beyond'))
if before > after:
    print("Before.")
else:
    print("After.")

After.


### 6. How many ratings each hour are above the average rating for exhibition 4?

In [8]:
sql_query = """
            SELECT COUNT(*), EXTRACT(HOUR FROM rating_instance.rating_created_at) AS hour
            FROM rating_instance
            JOIN rating_type
            ON rating_type.rating_type_id = rating_instance.rating_type_id
            WHERE rating_type_value > (
                SELECT AVG(rating_type_value) FROM rating_instance
                JOIN rating_type
                ON rating_type.rating_type_id = rating_instance.rating_type_id
                JOIN exhibition
                ON exhibition.exhibition_id = rating_instance.exhibition_id
                WHERE exhibition.exhibition_id = 5
            )
            GROUP BY EXTRACT(HOUR FROM rating_instance.rating_created_at)
            ;
            """

data = run_sql_query(conn, sql_query)
for time in data:
    print(f"{time.get('count')} ratings were above average at {time.get('hour')}:00.")


117 ratings were above average at 14:00.
37 ratings were above average at 18:00.
123 ratings were above average at 11:00.
139 ratings were above average at 10:00.
125 ratings were above average at 17:00.
110 ratings were above average at 12:00.
146 ratings were above average at 13:00.
122 ratings were above average at 16:00.
98 ratings were above average at 15:00.
129 ratings were above average at 9:00.
66 ratings were above average at 8:00.


### 7. Do Zoology exhibitions get better ratings than other types?

In [9]:
sql_query = """
            SELECT d.department_id, d.department_name, SUM(rt.rating_type_value) AS total_ratings
            FROM department AS d
            JOIN exhibition AS e
            ON d.department_id = e.department_id
            JOIN rating_instance AS ri
            ON ri.exhibition_id = e.exhibition_id
            JOIN rating_type AS rt
            ON rt.rating_type_id = ri.rating_type_id
            GROUP BY d.department_id
            ORDER BY total_ratings
            DESC
            ;
            """

data = run_sql_query(conn, sql_query)
if data[0].get('department_name') == 'Zoology':
    print('Yes.')

Yes.


### 8. What exhibition has the most emergencies?

In [10]:
sql_query = """
            SELECT e.exhibition_name, SUM(CASE WHEN st.support_description = 'Emergency' THEN 1 ELSE 0 END) AS emergency_count
            FROM exhibition AS e
            JOIN support_instance AS si
            ON e.exhibition_id = si.exhibition_id
            JOIN support_type AS st
            ON si.support_type_id = st.support_type_id
            GROUP BY exhibition_name
            ORDER BY emergency_count
            DESC
            ;
            """
data = run_sql_query(conn, sql_query)
print(f"{data[0].get('exhibition_name')} has the most emergencies, with {data[0].get('emergency_count')} emergencies.")

The Crenshaw Collection has the most emergencies, with 2 emergencies.


### 9. Which exhibitions receive fewer assistance requests than the average?

In [11]:
sql_query = """
            WITH exh_assistances AS
                (SELECT e.exhibition_name, COUNT(*) as assistance_count
                FROM exhibition AS e
                JOIN support_instance AS si
                    ON si.exhibition_id = e.exhibition_id
                JOIN support_type AS st
                    ON st.support_type_id = si.support_type_id
                WHERE st.support_type_value = 0
                GROUP BY e.exhibition_name),
            average_requests_count AS
                (
                    SELECT ROUND(AVG(assistance_count), 0) as total_average
                    FROM exh_assistances
                )
            SELECT ea.exhibition_name,
                CASE WHEN
                    assistance_count < total_average THEN true ELSE false END AS fewer
                FROM exh_assistances AS ea
                CROSS JOIN average_requests_count;
            """

data = run_sql_query(conn, sql_query)
for exhibition in data:
    if exhibition.get('fewer') == True:
        print(exhibition.get('exhibition_name'))

Adaptation
Measureless to Man
Our Polluted World
The Crenshaw Collection


### 10. Are there particular times when assistance requests/emergencies are more likely?

In [12]:
sql_query = """
            SELECT COUNT(*), EXTRACT(HOUR FROM support_instance.instance_created_at) AS hour
            FROM support_instance
            JOIN support_type
            ON support_type.support_type_id = support_type.support_type_id
            WHERE support_type_value > (
                SELECT AVG(support_type_value) FROM support_instance
                JOIN support_type
                ON support_type.support_type_id = support_instance.support_type_id
                JOIN exhibition
                ON exhibition.exhibition_id = support_instance.exhibition_id
            )
            GROUP BY EXTRACT(HOUR FROM support_instance.instance_created_at)
            ;
            """

data = run_sql_query(conn, sql_query)
times = [str(time.get('hour')) + ":00" for time in data]
times

['14:00',
 '17:00',
 '18:00',
 '11:00',
 '15:00',
 '12:00',
 '9:00',
 '13:00',
 '16:00',
 '10:00',
 '8:00']

### 11. Are emergencies more likely in exhibitions that are more exciting than others?

In [35]:
sql_query = """
            WITH exhibition_excitement AS (
                SELECT e.exhibition_id, e.exhibition_name, ROUND(AVG(rt.rating_type_value), 2),
            (CASE WHEN AVG(rt.rating_type_value) > (
                SELECT AVG(rt.rating_type_value)
                FROM rating_instance AS ri
                LEFT JOIN rating_type AS rt
                ON (ri.rating_type_id = rt.rating_type_id)
            ) THEN 'more_exciting' ELSE 'less_exciting' END) AS excitement
            FROM rating_instance AS ri
            LEFT JOIN exhibition AS e
            ON (e.exhibition_id = ri.exhibition_id)
            LEFT JOIN rating_type AS rt
            ON (rt.rating_type_id = ri.rating_type_id)
            GROUP BY e.exhibition_id, e.exhibition_name
            ),

            emergency_count AS (
                SELECT e.exhibition_id, COUNT(*) AS count
                FROM exhibition AS e
                LEFT JOIN support_instance AS si
                ON (e.exhibition_id = si.exhibition_id)
                LEFT JOIN support_type AS st
                ON (st.support_type_id = si.support_type_id)
                WHERE st.support_description = 'Emergency'
                GROUP BY e.exhibition_id
            )

            SELECT ee.exhibition_name, ee.excitement, COALESCE(ec.count, 0) AS ec_count
            FROM exhibition_excitement AS ee
            LEFT JOIN emergency_count AS ec
            ON ee.exhibition_id = ec.exhibition_id
            ;
            """

data = run_sql_query(conn, sql_query)
less_exciting = [exhibit for exhibit in data if exhibit.get(
    'excitement') == 'less_exciting' and int(exhibit.get('ec_count')) > 0]
more_exciting = [exhibit for exhibit in data if exhibit.get(
    'excitement') == 'more_exciting' and int(exhibit.get('ec_count')) > 0]
if len(less_exciting) > len(more_exciting):
    print('No.')
else:
    print('Yes.')

No.


### 12. Which floors are above average for ratings?

In [36]:
sql_query = """
            WITH floor_average_rating AS (
                SELECT f.floor, AVG(rt.rating_type_value) AS avg_rating
                FROM floor AS f
                JOIN exhibition AS e
                    ON f.floor_id = e.floor_id
                JOIN rating_instance AS ri
                    ON e.exhibition_id = ri.exhibition_id
                JOIN rating_type AS rt
                    ON rt.rating_type_id = ri.rating_type_id
                GROUP BY
                    f.floor
            )
            SELECT far.floor, ROUND(far.avg_rating, 2) AS floor_rating
            FROM floor_average_rating AS far
            WHERE far.avg_rating > (SELECT AVG(avg_rating) FROM floor_average_rating)
            ;
            """

data = run_sql_query(conn, sql_query)
for floor in data:
    print(f"Floor {floor.get('floor')} is above average rating, at a rating of {floor.get('floor_rating')}.")

Floor Vault is above average rating, at a rating of 1.93.
Floor 1 is above average rating, at a rating of 2.47.


## End

Close the connection to the database.

In [28]:
conn.rollback()
conn.close()